In [ ]:
import datetime
import threading
from os.path import dirname

from typing_extensions import List, Optional, Dict

from pycram.ros import logerr
from .datastructures.events import *
from .datastructures.object_tracker import ObjectTracker
from .detectors.coarse_event_detectors import *
from .episode_player import EpisodePlayer
from .event_logger import EventLogger
from .utils import check_if_object_is_supported, Imaginator


class EpisodeSegmenter(ABC):

    def __init__(self, episode_player: EpisodePlayer,
                 detectors_to_start: Optional[List[Type[DetectorWithStarterEvent]]] = None,
                 initial_detectors: Optional[List[Type[AtomicEventDetector]]] = None,
                 annotate_events: bool = False,
                 plot_timeline: bool = False,
                 show_plots: bool = False,
                 plot_save_path: Optional[str] = None):
        """
        Initializes the EpisodeSegmenter class.

        :param episode_player: The thread that plays the episode and generates the motion.
        :param detectors_to_start: The list of event detectors that should be started.
        :param annotate_events: A boolean value that indicates if the events should be annotated.
        :param plot_timeline: A boolean value that indicates if the events timeline should be plotted.
        :param show_plots: A boolean value that indicates if the plots should be shown.
        :param plot_save_path: The path where the plots should be saved.
        """
        self.episode_player: EpisodePlayer = episode_player
        self.detectors_to_start: List[Type[DetectorWithStarterEvent]] = detectors_to_start if detectors_to_start else []
        self.initial_detectors: List[Type[AtomicEventDetector]] = initial_detectors if initial_detectors else []
        self.logger = EventLogger(annotate_events, [detector.event_type() for detector in self.detectors_to_start + self.initial_detectors])
        self.objects_to_avoid = ['particle', 'floor', 'kitchen']
        self.starter_event_to_detector_thread_map: Dict[Tuple[Event, Type[DetectorWithStarterEvent]], DetectorWithStarterEvent] = {}
        self.detector_threads_list: List[EventDetectorUnion] = []
        self.object_trackers: Dict[Object, ObjectTracker] = {}
        self.plot_timeline = plot_timeline
        self.show_plots = show_plots
        self.plot_save_path = plot_save_path
        self.kill_event: threading.Event = threading.Event()

    def reset(self):
        for detector in self.starter_event_to_detector_thread_map.values():
            detector_str = str(detector)
            detector.reset()
            detector.stop()
            logerr(f"Detector {detector_str} stopped, joining it now...")
            detector.join()
            logerr(f"Joined {detector_str}")
            self.detector_threads_list.remove(detector)
        self.starter_event_to_detector_thread_map = {}
        initial_detector_instances = [detector for detector in self.detector_threads_list
                                      if type(detector) in self.initial_detectors]
        for detector in initial_detector_instances:
            detector_str = str(detector)
            detector.stop()
            logerr(f"Detector {detector_str} stopped, joining it now...")
            detector.join()
            logerr(f"Joined {detector_str}")
            self.detector_threads_list.remove(detector)
        for detector in self.detector_threads_list:
            detector_str = str(detector)
            detector.stop()
            logerr(f"Detector {detector_str} stopped, joining it now...")
            detector.join()
            logerr(f"Joined {detector_str}")
        self.detector_threads_list = []
        time.sleep(0.5)
        self.logger.reset()
        self.run_initial_event_detectors()

    def start(self) -> None:
        """
        Start the episode player and the event detectors.
        """
        self.start_episode_player_and_wait_till_ready()
        self.run_event_detectors()

    def stop(self) -> None:
        self.kill_event.set()

    def start_episode_player_and_wait_till_ready(self) -> None:
        """
        Start the Episode player thread, and waits until the thread signals that it is
        ready (e.g., the replay environment is initialized with all objects in starting poses).
        """
        if not self.episode_player.is_alive():
            self.episode_player.start()
        while not self.episode_player.ready:
            time.sleep(0.1)

    def run_event_detectors(self) -> None:
        """
        Run the event detectors on the motion generated by the motion generator thread.
        """
        self.create_detector_and_start_it(NewObjectDetector, avoid_objects=self.avoid_object)
        self.run_initial_event_detectors()
        closed_threads = False

        while (not closed_threads) or (self.logger.event_queue.unfinished_tasks > 0):
            if (not self.episode_player.is_alive() or self.kill_event.is_set()) and not closed_threads:
                time.sleep(0.5)
                # join all motion threads
                joined = []
                for detector_thread in self.detector_threads_list:
                    if isinstance(detector_thread, MotionDetector):
                        detector_thread.stop()
                        logdebug(f"Joining {detector_thread.thread_id}, {detector_thread.name}")
                        detector_thread.join()
                        joined.append(detector_thread)
                for detector_thread in self.detector_threads_list:
                    if detector_thread not in joined:
                        detector_thread.stop()
                        logdebug(f"Joining {detector_thread.thread_id}, {detector_thread.name}")
                        detector_thread.join()
                        joined.append(detector_thread)
                closed_threads = True

            next_event = self.logger.get_next_event()

            if next_event is None:
                time.sleep(0.01)
                continue

            self.process_event(next_event)

        if self.plot_timeline:
            self.logger.plot_events(show=self.show_plots, save_path=self.plot_save_path)

        self.join()

    def process_event(self, event: EventUnion) -> None:
        """
        Process the event generated by the event logger, and start the detector threads that are triggered by the event.

        :param event: The event that should be processed.
        """
        self._process_event(event)
        self.start_triggered_detectors(event)

    def start_triggered_detectors(self, event: EventUnion) -> None:
        """
        Start the detector threads that are triggered by the event.

        :param event: The event that triggers the detector threads.
        """
        for event_detector in self.detectors_to_start:
            if event_detector.start_condition_checker(event):
                self.start_detector_thread_for_starter_event(event, event_detector)

    @abstractmethod
    def _process_event(self, event: Event) -> None:
        """
        Process the event generated by the event logger.

        :param event: The event that should be processed.
        """
        pass

    def run_initial_event_detectors(self) -> None:
        """
        Run the initial event detectors on the episode played by the episode player thread.
        """
        for detector in self.initial_detectors:
            self.create_detector_and_start_it(detector)
        self._run_initial_event_detectors()
    
    @abstractmethod
    def _run_initial_event_detectors(self) -> None:
        """
        Run the initial event detectors on the episode played by the episode player thread.
        """
        pass

    def run_objects_initial_event_detectors(self) -> None:
        """
        Start the motion detection threads for the objects in the world, and imagine supports for objects
        that seem to be supported but have no actual support detected.
        """
        self.episode_player.pause()
        set_of_objects = set()
        for obj in World.current_world.objects:
            if not self.avoid_object(obj):
                set_of_objects.add(obj)
                # if not check_if_object_is_supported(obj):
                #     logdebug(f"Object {obj.name} is not supported.")
                #     if World.current_world.conf.supports_spawning:
                #         Imaginator.imagine_support_for_object(obj)
                #         logdebug(f"Imagined support for object {obj.name}.")
        for obj in set_of_objects:
            self.start_motion_threads_for_object(obj)
            self.start_contact_threads_for_object(obj)
        self.episode_player.resume()

    def update_tracked_objects(self, event: EventUnion) -> None:
        """
        Update the tracked objects based on the event, for example a contact event would reveal new objects that should
        be tracked when an already tracked object comes into contact with a new object.

        :param event: The event that was triggered.
        """
        involved_objects = self.get_involved_bodies(event)
        if not involved_objects:
            return
        logdebug(f"Involved objects: {[obj.name for obj in involved_objects]}")
        for obj in involved_objects:
            if self.avoid_object(obj):
                continue
            if isinstance(obj, Link) and obj.parent_entity in self.object_trackers.keys():
                continue
            if obj not in self.object_trackers.keys():
                logdebug(f"New object {obj.name}")
                self.object_trackers[obj] = ObjectTracker(obj)
                self.start_tracking_threads_for_new_object_and_event(obj, event)

    @abstractmethod
    def start_tracking_threads_for_new_object_and_event(self, new_object: Object, event: EventUnion):
        """
        Start the tracking threads for the new object, these threads are used to track the object's motion or contacts
         for example.

        :param new_object: The new object that should be tracked.
        :param event: The event that triggered the tracking.
        """
        pass

    def get_involved_bodies(self, event: EventUnion) -> List[PhysicalBody]:
        """
        Get the bodies involved in the event.

        :param event: The event that involves the objects.
        :return: A list of Object instances that are involved in the event.
        """
        if isinstance(event, EventWithTrackedObjects):
            return event.involved_bodies

    def avoid_object(self, obj: Object) -> bool:
        """
        Check if the object should be avoided.

        :param obj: The object to check.
        :return: True if the object should be avoided, False otherwise.
        """
        return ((obj.is_an_environment or issubclass(obj.ontology_concept, (Supporter, Location)) or
                 any([k in obj.name.lower() for k in self.objects_to_avoid])) or
                (isinstance(obj, Link) and self.avoid_object(obj.parent_entity)))

    def start_motion_threads_for_object(self, obj: Object, event: Optional[NewObjectEvent] = None) -> None:
        """
        Start the motion detection threads for the object.

        :param obj: The Object instance for which the motion detection threads are started.
        :param event: The NewObjectEvent instance that represents the creation of the object.
        """
        for detector in (TranslationDetector,): #RotationDetector):
            self.create_detector_and_start_it(detector, tracked_object=obj, starter_event=event,
                                              time_between_frames=self.time_between_frames)

    def start_contact_threads_for_object(self, obj: Object,
                                         event: Optional[ContactEvent] = None) -> None:
        """
        Start the contact threads for the object and updates the tracked objects.

        :param obj: The Object instance for which the contact threads are started.
        :param event: The ContactEvent instance that represents the contact event with the object.
        """
        for detector in (ContactDetector, LossOfContactDetector):
            self.create_detector_and_start_it(detector, tracked_object=obj, starter_event=event)

    def start_detector_thread_for_starter_event(self, starter_event: EventUnion,
                                                detector_type: TypeEventDetectorUnion):
        """
        Start the detector thread for the given starter event.

        :param starter_event: The event that starts the detector thread.
        :param detector_type: The type of the detector.
        """
        if not self.is_detector_redundant(detector_type, starter_event):
            if detector_type == PlacingDetector:
                logdebug(f"new placing detector for object {starter_event.tracked_object.name}")
            self.create_detector_and_start_it(detector_type, starter_event=starter_event)

    @staticmethod
    def ask_now(case_dict):
        detector_type = case_dict['detector_type']
        starter_event = case_dict['starter_event']
        self_ = case_dict['self_']
        output_ = case_dict['output_']
        if issubclass(detector_type, GeneralPickUpDetector):
            if isinstance(starter_event, LossOfContactEvent):
                pick_up_detectors = [detector for (_, _), detector in self_.starter_event_to_detector_thread_map.items()
                                     if isinstance(detector, GeneralPickUpDetector)]
                if len(pick_up_detectors) > 0 and not output_:
                    return True
        return False
    redundant_detector_rdr = RDRDecorator(f"{dirname(__file__)}/rdrs", (bool,), True,
                                          fit=False, fitting_decorator=EpisodePlayer.pause_resume, ask_now=ask_now)
    @redundant_detector_rdr.decorator
    def is_detector_redundant(self, detector_type: TypeEventDetectorUnion, starter_event: EventUnion) -> bool:
        """
        Check if the detector is redundant.

        :param detector_type: The type of the detector.
        :param starter_event: The event that starts the detector thread.
        :return: True if the detector is redundant, False otherwise.
        """
        if (starter_event, detector_type) in self.starter_event_to_detector_thread_map.keys():
            detector = self.starter_event_to_detector_thread_map[(starter_event, detector_type)]
            if detector.is_alive() or (detector.detected_before and detector.run_once):
                return True
        return False

    def create_detector_and_start_it(self, detector_type: TypeEventDetectorUnion,
                                     tracked_object: Optional[Object] = None,
                                     starter_event: Optional[EventUnion] = None,
                                     *detector_args, **detector_kwargs) -> None:
        """
        Start and add an event detector to the detector threads.

        :param detector_type: The event detector to be started and added.
        :param tracked_object: The object to be tracked by the detector.
        :param starter_event: The event that starts the detector thread.
        :param detector_args: The positional arguments to be passed to the detector constructor.
        :param detector_kwargs: The keyword arguments to be passed to the detector constructor.
        """
        detector_kwargs = self.get_detector_args(detector_type, tracked_object=tracked_object,
                                               starter_event=starter_event, **detector_kwargs)
        detector_kwargs['episode_player'] = self.episode_player
        detector_kwargs['logger'] = self.logger
        detector = detector_type(**detector_kwargs)
        self.starter_event_to_detector_thread_map[(starter_event, detector_type)] = detector
        self.start_and_add_detector(detector)

    def start_and_add_detector(self, detector: EventDetectorUnion) -> None:
        """
        Start and add an event detector to the detector threads.

        :param detector: The event detector to be started and added.
        """
        detector.start()
        self.detector_threads_list.append(detector)
        logdebug(f"Created {type(detector).__name__}")
        if isinstance(detector, DetectorWithStarterEvent) and detector.starter_event is not None:
            logdebug(f"For starter event {detector.starter_event}")

    @staticmethod
    def get_detector_args(detector_type: TypeEventDetectorUnion, tracked_object: Optional[Object] = None,
                          starter_event: Optional[EventUnion] = None, **other_detector_kwargs):
        """
        Get the detector arguments from the tracked object and/or the starter event.

        :param detector_type: The type of the detector.
        :param tracked_object: The object to be tracked by the detector.
        :param starter_event: The event that starts the detector thread.
        :param other_detector_args: The positional arguments to be passed to the detector constructor.
        """
        tracked_object = tracked_object if tracked_object is not None else \
            getattr(starter_event, 'tracked_object', None)
        if tracked_object and issubclass(detector_type, (DetectorWithTrackedObject, DetectorWithTwoTrackedObjects)):
            other_detector_kwargs['tracked_object'] = tracked_object
        if starter_event and issubclass(detector_type, DetectorWithStarterEvent):
            other_detector_kwargs['starter_event'] = starter_event
        return other_detector_kwargs

    @property
    def time_between_frames(self) -> datetime.timedelta:
        """
        :return: The time between frames of the episode player.
        """
        return self.episode_player.time_between_frames

    def join(self):
        """
        Join all the threads.
        """
        self.logger.print_events()
        self.logger.join()
        logdebug("All threads joined.")


class AgentEpisodeSegmenter(EpisodeSegmenter):
    """
    The AgentBasedEpisodeSegmenter class is used to segment motions into activities (e.g. PickUp) by tracking the
     events that are relevant to the agent for example contact events of the hands or robot.
    """

    def start_tracking_threads_for_new_object_and_event(self, new_object: Object, event: Optional[ContactEvent] = None):
        logdebug(f"Creating contact and motion threads for object {new_object.name}")
        self.start_contact_threads_for_object(new_object, event)
        self.start_motion_threads_for_object(new_object, event)

    def _process_event(self, event: Event) -> None:
        if isinstance(event, ContactEvent):
            self.update_tracked_objects(event)

    def _run_initial_event_detectors(self) -> None:
        """
        Start the contact threads for the agents.
        """
        agents = self.get_agents()
        for agent in agents:
            self.object_trackers[agent] = ObjectTracker(agent)
            self.start_contact_threads_for_object(agent)

    @staticmethod
    def get_agents() -> List[Object]:
        """
        :return: A list of Object instances that represent the available agents in the world.
        """
        return [obj for obj in World.current_world.objects if issubclass(obj.obj_type, Agent)]


class NoAgentEpisodeSegmenter(EpisodeSegmenter):
    """
    The NoAgentEpisodeSegmenter class is used to segment episodes into activities (e.g. PickUp) by tracking the
     events that are relevant to the objects in the world with the lack of an agent in the episode.
    """

    def __init__(self, episode_player: EpisodePlayer,
                 detectors_to_start: Optional[List[Type[DetectorWithStarterEvent]]] = None,
                 annotate_events: bool = False, **kwargs):
        # if detectors_to_start is None:
        #     detectors_to_start = [GeneralPickUpDetector, PlacingDetector]
        super().__init__(episode_player, detectors_to_start=detectors_to_start, annotate_events=annotate_events,
                         **kwargs)

    def start_tracking_threads_for_new_object_and_event(self, new_object: Object, event: EventUnion):
        pass

    def get_involved_bodies(self, event: EventUnion) -> List[Object]:
        return []

    def _process_event(self, event: EventUnion) -> None:
        pass

    def _run_initial_event_detectors(self) -> None:
        """
        Start the motion detection threads for the objects in the world.
        """
        self.run_objects_initial_event_detectors()
